In [1]:
%load_ext autoreload
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

print('findspark initialized ...')

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.appName('mlonspark')\
    .config('spark.executor.instances', '8')\
    .config('spark.jars', '/opt/dev/target/ml-on-spark-1.0.jar')\
    .getOrCreate()

#   
    
print('pyspark ready ...')

env: SPARK_HOME=/usr/hdp/current/spark2-client
findspark initialized ...
pyspark ready ...


In [2]:
train = spark.read.format("parquet").load("/data/books/ratings-train.parquet")
train.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [3]:
%autoreload
from mlonspark.alternating_least_square import AlternatingLeastSquare
from pyspark.ml.recommendation import ALS
#from mlonspark.alternating_least_square import XALS

als = AlternatingLeastSquare()\
    .setUserCol("userId")\
    .setItemCol("bookId")\
    .setRatingCol("rating")\
    .setMaxIter(10)

#\
#    .setImplicitPrefs(True)

print(als)

AlternatingLeastSquare_4b4db04e24c73ff6f966


In [4]:
model = als.fit(train)
print(model)

AlternatingLeastSquare_4b4db04e24c73ff6f966


In [6]:
model.write().overwrite().save('/data/books/model.alsmodel')

In [6]:
%autoreload
#from mlonspark.alternating_least_square import AlternatingLeastSquareModel

#model = AlternatingLeastSquareModel.read().load('/data/books/model.alsmodel')

In [7]:
print(model)

AlternatingLeastSquare_4235bce2c3f8585e3432


In [5]:
test = spark.read.format("parquet").load("/data/books/ratings-test.parquet")
test.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)



In [6]:
predictions = model.transform(test)
predictions.printSchema()
predictions.show(10)

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- bookId: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = false)

+-------+----------+-----------+------+------+--------------------+----+------+------------+
|User-ID|      ISBN|Book-Rating|rating|bookId|            Location| Age|userId|  prediction|
+-------+----------+-----------+------+------+--------------------+----+------+------------+
|  11676|0380778556|          0|   1.0|   148|       n/a, n/a, n/a|NULL| 11676|  0.72542936|
| 259312|0380778556|         10|   4.0|   148|missouri city, te...|NULL|259312|0.0020416633|
| 216882|0380778556|          9|   3.0|   148|adelaide, south a...|NULL|216882| 0.013926785|
| 130474|0380778556|          0|   1.0|   148|nashville, tennes...|  33|130474| 0.0173

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import isnan

evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("rating")\
    .setPredictionCol("prediction")

predictionsFiltered = predictions.where(~isnan(col("prediction")))
                                
print("number of predictions = %i" % predictionsFiltered.count())

rmse = evaluator.evaluate(predictionsFiltered)

print("RMSE = %f" % rmse)

number of predictions = 279344
RMSE = 1.617414


In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.recommendation import ALS

#als2 = XALS()\
#    .setUserCol("userId")\
#    .setItemCol("bookId")\
#    .setRatingCol("rating")

pipeline = Pipeline().setStages([als])

params = ParamGridBuilder()\
    .addGrid(als.rank, range(6, 24, 2))\
    .build()
#    .addGrid(als.rank, [2, 5, 10])\
    

cv = CrossValidator()\
    .setEstimator(als)\
    .setEstimatorParamMaps(params)\
    .setEvaluator(evaluator)\
    .setNumFolds(5)

model = cv.fit(train)

In [15]:
model.bestModel
model.avgMetrics

[1.6209561629874365,
 1.6205474326224263,
 1.6197064651447595,
 1.6194483412184253,
 1.619152001040322,
 1.6187775703541334,
 1.6186348739480114,
 1.618588836144526,
 1.6183966092557387]

In [16]:
bestModel = model.bestModel

bestModel.write().overwrite().save('/data/books/bestModel-rank.model')

In [17]:
predictionsBestModel = bestModel.transform(test)
bestModelRmse = evaluator.evaluate(predictionsBestModel)

print("RMSE = %f" % bestModelRmse)

RMSE = 1.619105
